#### Notebook example to run EMR serverless job from Sagemaker

us-east-1 applications: <br>
* pd-autoencoder-ad-v1 : 00f64bef5869kl09
* pd-autoencoder-ad-v2 : 00f66ohicnjchu09
* pd-autoencoder-ad-v3 : 00f6959e87i48609
<br>

us-west-2 applications: <br>
* pd-autoencoder-ad-container-v1


Note: while launching your job, please make note of the region from where you are running it.
jobs for us-east-region can only be launched from us-east-1 

In [ ]:
pwd

In [ ]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

In [ ]:
pip install -r requirements.txt

In [ ]:
pip install tensorflow

In [ ]:
pip install 
import tensorflow

In [3]:
from eks_ml_pipeline import EMRServerless

In [28]:
# application id
application_id = '00f66ohicnjchu09'
# pass job_run_id only when a job is running else it can be empty
job_run_id = "00f6a3pmvv5o7o09"
# don't chnage the serverless_job_role_arn
serverless_job_role_arn = 'arn:aws:iam::064047601590:role/hamza-emr-serverless-role'
# s3 bukcet name where the dependencies, logs and code sits
s3_bucket_name = 'emr-serverless-output-pd'
emr_emtry_point = 's3://emr-serverless-output-pd/code/pyspark/pd-autoencoder-ad/s3_test_emr.py'
zipped_env_path = 's3://emr-serverless-output-pd/code/pyspark/pd-autoencoder-ad/pyspark_deps_all_rec_types_v2.tar.gz'

In [29]:
emr_serverless = EMRServerless(application_id,job_run_id)

In [22]:
print("Starting EMR Serverless Spark App")
#emr_serverless.create_application("pd-autoencoder-test-emr-cli", "emr-6.6.0")
emr_serverless.start_application(application_id)
print(emr_serverless)

Starting EMR Serverless Spark App
EMR Serverless SPARK Application: 00f66ohicnjchu09


Below cell shows an example to run a new job

In [ ]:
# Run (and wait for) a Spark job
print("Submitting new Spark job")
job_run_id = emr_serverless.run_spark_job(
    script_location=emr_entry_point,
    job_role_arn=serverless_job_role_arn,
    application_id = application_id,
    arguments=[f"s3://{s3_bucket_name}/emr-serverless/output"],
    s3_bucket_name=s3_bucket_name,
    zipped_env_path = zipped_env_path
)

In [30]:
emr_serverless.get_job_run()

{'applicationId': '00f66ohicnjchu09',
 'jobRunId': '00f6a3pmvv5o7o09',
 'arn': 'arn:aws:emr-serverless:us-east-1:064047601590:/applications/00f66ohicnjchu09/jobruns/00f6a3pmvv5o7o09',
 'createdBy': 'arn:aws:sts::064047601590:assumed-role/AmazonSageMakerServiceCatalogProductsUseRole/SageMaker',
 'createdAt': datetime.datetime(2022, 12, 13, 22, 16, 29, 642000, tzinfo=tzlocal()),
 'updatedAt': datetime.datetime(2022, 12, 14, 10, 16, 35, 534000, tzinfo=tzlocal()),
 'executionRole': 'arn:aws:iam::064047601590:role/hamza-emr-serverless-role',
 'state': 'CANCELLED',
 'stateDetails': 'The job execution time has reached the timeout and has been cancelled.',
 'releaseLabel': 'emr-6.9.0',
 'configurationOverrides': {'monitoringConfiguration': {'s3MonitoringConfiguration': {'logUri': 's3://emr-serverless-output-pd/logs/'}}},
 'jobDriver': {'sparkSubmit': {'entryPoint': 's3://emr-serverless-output-pd/code/pyspark/pd-autoencoder-ad/emr_job_runner_pod_final.py',
   'sparkSubmitParameters': '--conf sp

In [24]:
emr_serverless.fetch_driver_log(s3_bucket_name)

File output from stdout.gz:
----
 2022-12-13 22:17:25.610274: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 22:17:25.725055: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-13 22:17:25.728145: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/li

In [ ]:
# job_status = emr_serverless.get_job_run(job_run_id)
# print(f"Job finished: {job_run_id}, status is: {job_status.get('state')}")

# Fetch and print the logs
spark_driver_logs = emr_serverless.fetch_driver_log(s3_bucket_name)
print("File output from stdout.gz:\n----\n", spark_driver_logs, "\n----")

# Now stop and delete your application
#print("Stopping App")
#emr_serverless.stop_application()
#emr_serverless.delete_application()
print("Done! 👋")